In [14]:
import pandas as pd
import time
import dash
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from dash import dcc, html, Output, Input, State
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from dash import dcc, html, Input, Output, State

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("MusicDashboard") \
    .config("spark.jars", "jars/mysql-connector-java-8.4.0.jar") \
    .getOrCreate()

# Function to retrieve fan counts with error handling
def get_fan_counts():
    try:
        # Fetch data using Spark
        df = spark.read.format("jdbc") \
            .option("url", "jdbc:mysql://127.0.0.1:3306/starmeter_sim") \
            .option("dbtable", "user_dynamic_preferences") \
            .option("user", "root") \
            .option("password", "zipcode123") \
            .option("driver", "com.mysql.cj.jdbc.Driver") \
            .load()

        # Perform the aggregation using Spark
        fan_counts_df = df.groupBy("current_favorite").count().withColumnRenamed("count", "fan_count")

        # Convert to Pandas DataFrame for Dash processing
        fan_counts_pd = fan_counts_df.toPandas()

        if fan_counts_pd.empty:
            print("Warning: The retrieved DataFrame is empty.")
        return fan_counts_pd

    except Exception as e:
        print(f"Error retrieving data: {e}")
        return pd.DataFrame(columns=['current_favorite', 'fan_count'])

In [15]:


# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define custom styles for KPI cards
card_styles = [
    {"backgroundColor": "#0075A4", "color": "white", "padding": "0px", "borderRadius": "10px", "width": "200px", "height": "60px", "lineHeight": "1"},
    {"backgroundColor": "#008FAD", "color": "white", "padding": "0px", "borderRadius": "10px", "width": "200px", "height": "60px", "lineHeight": "1"},
    {"backgroundColor": "#00A697", "color": "white", "padding": "0px", "borderRadius": "10px", "width": "200px", "height": "60px", "lineHeight": "1"},
    {"backgroundColor": "#1EB769", "color": "white", "padding": "0px", "borderRadius": "10px", "width": "200px", "height": "60px", "lineHeight": "1"}
]

# Initialize layout with KPI cards next to each other and graph with legend on top
app.layout = html.Div([
     dbc.Row([
        dbc.Col(html.Button('Start', id='start-stop-button', n_clicks=0, className="btn btn-primary"), width="auto"),
        dbc.Col(html.H1("StarMeter", style={"fontSize": "24px"}), width=True)
    ], align='center', className="mb-4"),
    
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Fans (in millions)", className="card-title", style={"fontSize": "12px", "margin": "0"}),
                html.H5(id='total-fans', className="card-text", style={"fontSize": "16px", "margin": "0"})
            ])
        ], style=card_styles[0], className="mb-2"), width=2),
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Most Popular Celebrity", className="card-title", style={"fontSize": "12px", "margin": "0"}),
                html.H5(id='most-popular-celebrity', className="card-text", style={"fontSize": "16px", "margin": "0"})
            ])
        ], style=card_styles[1], className="mb-2"), width=2),
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Average Fans", className="card-title", style={"fontSize": "12px", "margin": "0"}),
                html.H5(id='average-fans', className="card-text", style={"fontSize": "16px", "margin": "0"})
            ])
        ], style=card_styles[2], className="mb-2"), width=2),
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Events", className="card-title", style={"fontSize": "12px", "margin": "0"}),
                html.H5(id='total-events', className="card-text", style={"fontSize": "16px", "margin": "0"})
            ])
        ], style=card_styles[3], className="mb-2"), width=2),
    ], className="g-1"),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='live-update-graph'), width=8),
        dbc.Col(html.Div(id='event-log', children="Event log will be displayed here.", style={"height": "400px", "overflowY": "auto", "border": "1px solid #ddd", "padding": "10px"}), width=4)
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='bar-chart'), width=12)
    ]),
    
    dcc.Interval(
        id='interval-component',
        interval=1000,  # update interval milliseconds
        n_intervals=0,
        disabled=True  # interval initially disabled
    ),
])

# Initialize data storage
fan_counts_data = {celebrity: [] for celebrity in ['Sabrina Carpenter', 'Snoop Dogg', 'Tony Stark', 'LeBron James']}
time_data = []

@app.callback(
    [Output('live-update-graph', 'figure'),
     Output('bar-chart', 'figure'),
     Output('total-fans', 'children'),
     Output('most-popular-celebrity', 'children'),
     Output('average-fans', 'children'),
     Output('total-events', 'children')],
    Input('interval-component', 'n_intervals')
)
def update_graph_and_kpis(n):
    global time_data, fan_counts_data

    try:
        # Fetch latest fan counts using Spark
        df = get_fan_counts()  # Replace with your data fetching function

        # Initialize KPI variables
        total_fans = 0
        most_popular_celeb = None
        max_fans = 0
        total_events = 0
        current_fan_counts = []  # To store the latest fan counts for the bar chart
        colors = ['#0075A4', '#008FAD', '#00A697', '#1EB769']

        if not df.empty:
            # Append current timestamp
            current_time = time.time()
            time_data.append(current_time - time_data[0] if time_data else 0)

            # Update fan counts data
            for idx, celebrity in enumerate(fan_counts_data.keys()):
                # Fetch the latest fan count for each celebrity
                fan_count = df[df['current_favorite'] == celebrity]['fan_count'].sum() if not df[df['current_favorite'] == celebrity].empty else 0
                fan_counts_data[celebrity].append(fan_count)
                current_fan_counts.append(fan_count)  # Add to current fan counts for bar chart
                total_events += len(fan_counts_data[celebrity])

                # Update total fans and most popular celebrity
                total_fans += fan_count
                if fan_count > max_fans:
                    max_fans = fan_count
                    most_popular_celeb = celebrity
            colors = [
                '#1f77b4',  # muted blue
                '#9467bd',  # muted purple
                '#d62728',  # brick red
                '#17becf'   # blue-teal
            ]
            
            # Create traces for each celebrity in the line chart
            line_fig = go.Figure()
            for idx, (celebrity, fan_counts) in enumerate(fan_counts_data.items()):
                line_fig.add_trace(go.Scatter(
                    x=time_data,
                    y=fan_counts,
                    mode='lines+markers',
                    name=celebrity,
                    marker=dict(size=5, color=colors[idx]),
                    line=dict(color=colors[idx]),
                    text=['Event happened here'] * len(fan_counts),
                    hoverinfo='text',
                    customdata=list(range(len(fan_counts))),  # Index of the markers
                ))

            # Ensure time_data has enough data points to set range
            if len(time_data) > 1:
                line_fig.update_layout(
                    xaxis=dict(range=[max(time_data) - 100, max(time_data)]),
                )

            line_fig.update_layout(
                xaxis_title='Time (seconds)',
                yaxis_title='Number of Fans (in millions)',
                margin=dict(l=0, r=0, t=40, b=0),  # Adjust margins to fit the layout
                autosize=True,
                showlegend=True,
                legend=dict(
                    orientation='h',  # Horizontal orientation
                    yanchor='bottom',
                    y=1.15,
                    xanchor='right',
                    x=1
                ),
                modebar=dict(
                    remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale2d', 'lasso2d', 'zoom2d', 'resetScale', 'toImage', 'plotly_logo']
                )
            )

           # Create bar chart for current fan counts
            bar_fig = go.Figure(data=[
                go.Bar(
                    x=list(fan_counts_data.keys()),
                    y=current_fan_counts,  # Use the latest counts
                    marker_color=colors,  # Match colors with the line graph
                    width=0.4  # Set the bar width; adjust as needed
                )
            ])
            # Set the y-axis range to the total current fans
            bar_fig.update_layout(
                title='Current Fans per Celebrity',
                xaxis_title='Celebrity',
                yaxis_title='Current Fans',
                yaxis=dict(range=[0, (total_fans/2)-1000]),  # Set range based on total fans
                margin=dict(l=40, r=40, t=40, b=80),  # Adjust margins to fit the layout
                autosize=True,
                width=400,  # Set the desired width
                height=400,  # Set the desired height
                showlegend=False,
                modebar=dict(
                remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale2d', 'lasso2d', 'zoom2d', 'resetScale', 'toImage', 'plotly_logo']
                )
            )
            bar_fig.update_xaxes(tickangle=-45)  # Rotate x-axis labels if needed

        

        else:
            line_fig = go.Figure().update_layout(
                title='No data available',
                xaxis_title='Time (seconds)',
                yaxis_title='Number of Fans'
            )
            bar_fig = go.Figure().update_layout(
                title='No data available',
                xaxis_title='Celebrity',
                yaxis_title='Current Fans'
            )

        average_fans = total_fans // len(fan_counts_data) if fan_counts_data else 0
        return line_fig, bar_fig, f'{total_fans}', most_popular_celeb or 'No data', f'{average_fans}', f'{total_events}'

    except Exception as e:
        print(f"Error in updating graph and KPIs: {e}")
        return go.Figure().update_layout(title="Error updating graph"), go.Figure().update_layout(title="Error updating graph"), 'Error', 'Error', 'Error', 'Error'


# Callback control start stop functionality
@app.callback(
    [Output('interval-component', 'disabled'),
     Output('start-stop-button', 'children')],
    [Input('start-stop-button', 'n_clicks')],
    [State('interval-component', 'disabled')]
)

def toggle_interval(n_clicks, is_disabled):
    # Toggle state interval (start stop updates)
    if n_clicks % 2 == 0:
        return True, 'Start'  # disabled, show 'Start' button text
    else:
        return False, 'Stop'  # enabled, show 'Stop' button text

if __name__ == '__main__':
    app.run_server(debug=True)
